In [1]:
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

import skimage.io as io
import pickle

import time

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras.models import Sequential
#for colab
# from tensorflow.keras.optimizers import Adam

from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from tensorflow.keras.layers import BatchNormalization,Dropout
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

from tensorflow.keras.layers import Layer
from keras.regularizers import l2
from keras import backend as K

from tensorflow.keras.applications.inception_v3 import InceptionV3
# from tensorflow.keras.preprocessing import image
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

from sklearn.metrics import confusion_matrix, classification_report


/home/asmaa/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/asmaa/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/asmaa/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/asmaa/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.

In [29]:
path="../Hieroglyphsdata_digital/"

In [34]:
def extractLAbel(file_name):
    name = file_name.replace(" ","")
    name = name.split(".")[0]
    label=name[0]
    for i in name[1:]:
        if i.isnumeric():
            label+=i
        else:
            break
    return name,label
    
# extractLAbel("A 23Dg5.png")

In [52]:
def loadData_digital(folderPictures=path):
    folders=next(os.walk(folderPictures))[1]
    img_groups = {}
    img_list={}
    for folder in folders:
        for img_file in os.listdir(folderPictures+folder):
            name, label = extractLAbel(img_file)
            if label in img_groups.keys():
                img_groups[label].append(folder+"__"+name)
            else:
                img_groups[label] = [folder+"__"+name]

            img_list[folder+"__"+name]=[label]
    for k,v in list(img_groups.items()):
        if len(v)==1: del img_groups[k]
     # Extract only N hieroglyph classes randomly

    nclass = len(img_groups.keys())


    dataHiero=pd.DataFrame.from_dict(img_list,orient='index')
    dataHiero.columns = ["label"]
    dataHiero = dataHiero[dataHiero.label != 'UNKNOWN']

#     dataHiero = dataHiero.loc[dataHiero['label'].isin(short_dico)]


    dataHiero.reset_index(level=0, inplace=True)

    return dataHiero,img_groups

In [147]:
def read_images(img_groups,path):
    X=[]
    y=[]
    glyph_sizes={}
    low=0
    for glyph in img_groups:
        category_images=[]
        high=low
        for img_path in img_groups[glyph] :
            try:
    #             print(img_path)
                folder,name = img_path.split('__')
                if os.path.exists(path+folder+'/'+name+'.png'):
                    image = io.imread(path+folder+'/'+name+'.png')
                elif os.path.exists(path+folder+'/'+name+'.tiff'):
                    image = io.imread(path+folder+'/'+name+'.tiff')
                else:
                    image = io.imread(path+folder+'/'+name+'.jpg')
                
                if len(image.shape) > 2:
                    #convert the image from RGBA2RGB
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#                     print(image.shape)
#                 image= cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#                 print(img_path)
#                 print(image.shape)
                image=cv2.resize(image, (50,75))
                X.append(image)
                y.append(glyph)
                high+=1
            except:
                print(img_path)
#         X.append(np.array(category_images))
        glyph_sizes[glyph]=(low,high-1)
        low=high
        
    return np.array(X),np.array(y).reshape((-1,1)),glyph_sizes
            
    

In [142]:
dataHiero,img_groups=loadData_digital(folderPictures=path)
dataHiero.head()

,index,label
0,Resources__J2,J2
1,Resources__F130,F130
2,Resources__Z2B,Z2
3,Resources__X4A,X4
4,Resources__F19,F19


In [148]:
dataHiero.shape

(5850, 2)

In [149]:
X,y,sizes=read_images(img_groups,path)

hieroglyphs__US22V78vara
hieroglyphs__codageenvoin°35


In [150]:
len(np.unique(y))
y.shape

(5002, 1)

In [152]:
X.shape

(5002, 75, 50)

In [154]:
# sizes

In [159]:
X,y = shuffle(X,y)

In [161]:
def get_sizes(X,Y):
    sizes={}
    for i ,(x,y) in enumerate(zip(X,Y)):
#         print(i,x,y)
        if y[0] in sizes:
            sizes[y[0]].append(i)
        else:
            sizes[y[0]]=[i]
    return sizes
    

In [162]:
sizes=get_sizes(X,y)

In [163]:
with open("train_digital.pickle", "wb") as f:
    pickle.dump((X,y,sizes),f)